In [11]:
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
	temperature=0.1,
	streaming=True,
	callbacks=[
		StreamingStdOutCallbackHandler()
	]
)
memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=120,
	return_messages=True,
)

examples = [
	{
		"movie_name": "탑건",
		"history": [],
		"answer": "🛩️👨‍✈️🔥",
	},
	{
		"movie_name": "대부",
		"history": [],
		"answer": "👨‍👨‍👦🔫🍝",	
	},
]

example_prompt = ChatPromptTemplate.from_messages([ 
	(
		"human",
		"{movie_name}"
	),
	(
		"ai",
		"{answer}"
	),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
	example_prompt=example_prompt,
	examples=examples
)

prompt = ChatPromptTemplate.from_messages([ 
	("system", "당신은 매우 뛰어난 영화 평론가입니다. 영화 제목을 들으면 그 영화에 대해 이모티콘 3개로 설명할 수 있습니다."),
	few_shot_prompt,
	MessagesPlaceholder(variable_name="history", ),
	("human", "{movie_name}"),
])

def load_memory(_):
	return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(movie_name):
	result = chain.invoke({"movie_name": movie_name})
	# print(result)

	memory.save_context(
		{"input": movie_name},
		{"output": result.content},
	)

In [12]:
invoke_chain("캡틴아메리카: 더 윈터 솔져")

🇺🇸❄️🛡️

In [13]:
invoke_chain("슬램덩크")

🏀🔥🤯

In [21]:
memory.load_memory_variables({})["history"]

[HumanMessage(content='캡틴아메리카: 더 윈터 솔져'),
 AIMessage(content='🇺🇸❄️🛡️'),
 HumanMessage(content='슬램덩크'),
 AIMessage(content='🏀🔥🤯')]

In [26]:
chain.invoke({"movie_name": "내가 방금 질문한 영화 이전에 질문한 영화 제목은 뭐지?"})

이전에 당신이 질문한 영화는 "캡틴 아메리카: 더 윈터 솔져"였습니다.

AIMessageChunk(content='이전에 당신이 질문한 영화는 "캡틴 아메리카: 더 윈터 솔져"였습니다.')

In [16]:
chain.invoke({"movie_name": "내가 마지막으로 물어본 영화 제목이 뭐였어?"})

마지막으로 물어보신 영화 제목은 "슬램덩크"였습니다.

AIMessageChunk(content='마지막으로 물어보신 영화 제목은 "슬램덩크"였습니다.')